# Reddit Post Analysis

1. **Sets the Role and Tone**  
   Instructs the AI to act as an **expert analyst** specializing in extracting insights from online forums like Reddit.

2. **Guides Sentiment Analysis**  
   Asks the AI to evaluate overall sentiment (e.g., positive, neutral, negative), and to present it as approximate percentages with a brief rationale.

3. **Groups and Labels Themes**  
   Instructs the AI to identify and cluster **key discussion themes**, perspectives, and emotional tones. Each theme should be explained and illustrated with **example comments**.

4. **Creates an Insights Table**  
   Requests a structured table with fields like *Perspectives, Frustrations, Tools, Suggestions* to concisely summarize the discussion’s core insights.

5. **Describes Community Dynamics**  
   Asks the AI to assess the **interaction style** (e.g., supportive, sarcastic, argumentative) and note any social patterns (e.g., consensus or conflict).

#### Imports

In [ ]:
import praw
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, Image
from openai import OpenAI

#### Load Credentials

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [ ]:
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
    username=os.getenv("REDDIT_USERNAME"),
    password=os.getenv("REDDIT_PASSWORD")
)

print("Authenticated as:", reddit.user.me())

In [ ]:
openai = OpenAI()

#### Reddit Post Scraper

In [ ]:

class RedditPostScraper:
    def __init__(self, url):
        self.submission = reddit.submission(url=url)
        self.submission.comments.replace_more(limit=None)
        self._title = self.submission.title
        self._text = self.submission.selftext
        self._comments = ""
        self._formatted_comments = []  # for reprocessing if needed

    def _generate_comments(self):
        comments_list = []
        for top_level in self.submission.comments:
            top_author = top_level.author.name if top_level.author else "[deleted]"
            comments_list.append(f"{top_author}: {top_level.body}")

            for reply in top_level.replies:
                reply_author = reply.author.name if reply.author else "[deleted]"
                comments_list.append(
                    f"{reply_author} replied to {top_author}'s comment: {reply.body}"
                )
        self._formatted_comments = comments_list

    def title(self):
        return f"Title:\n{self._title}\n{self._text}"

    def comments(self, max_words=None):
        if not self._formatted_comments:
            self._generate_comments()

        output_comments = []
        total_words = 0

        for comment in self._formatted_comments:
            word_count = len(comment.split())
            if max_words and total_words + word_count > max_words:
                break
            output_comments.append(comment)
            total_words += word_count

        return "Text:\n" + "\n\n".join(output_comments)

In [ ]:
# post = RedditPostScraper("https://www.reddit.com/r/running/comments/1l77osa/pushing_through_a_run/")
# print(post.title())
# print(post.comments(2000))

#### System and User Prompt

In [ ]:
system_prompt = '''You are an expert analyst specializing in extracting insights from online discussion forums. You will be given the title of a Reddit post and a list of comments (some with replies). Your task is to analyze the sentiment of the discussion and extract structured insights that reflect the collective responses.

Your response **must be in well-formatted Markdown**. Use clear section headers (`##`, `###`), bullet points, and tables where appropriate.

Perform the following tasks:

---

## 1. Overall Sentiment Breakdown

- Determine the overall sentiment of the responses (e.g., positive, negative, neutral, mixed).
- Express the sentiment as approximate percentages (e.g., 60% positive, 25% neutral, 15% negative).
- Provide a short explanation for why the sentiment skews this way, referring to tone, topic sensitivity, controversy, humor, or supportiveness.

---

## 2. Thematic Grouping of Comments

- Identify key recurring **themes, perspectives, or discussion threads** in the comments.
- For each theme, create a subheading.
- Under each:
  - Briefly describe the focus or tone of that cluster (e.g., personal stories, criticism, questions, jokes).
  - Include 1–2 **example comments** using quote formatting (`>`), preferably ones with replies or high engagement.

---

## 3. Insights Table

If applicable, extract and structure insights into the following table. Leave any column empty if it’s not relevant to the post type:

| Perspectives/ Motivations     | Pains/ Concerns/ Frustrations    | Tools / References / Resources       | Suggestions / Solutions            |
|-------------------------------|----------------------------------|--------------------------------------|------------------------------------|
| - ...                         | - ...                            | - ...                                | - ...                              |

- Populate this table with concise bullet points.
- Adapt categories to match the discussion type (e.g., switch "Suggestions" to "Reactions" if it's a news thread).

---

## 4. Tone and Community Dynamics

- Comment on the **style and culture** of interaction: humor, sarcasm, empathy, trolling, intellectual debate, etc.
- Mention any noticeable social dynamics: agreement/disagreement, echo chambers, respectful debate, or hostility.
- Include casual or emotional comments if they illustrate community personality.

---

**Respond only in well-formatted Markdown.** Structure your output for clarity and insight, suitable for rendering in documentation, reports, or dashboards. Do not summarize every comment — focus on patterns, perspectives, and collective signals.

'''

In [ ]:
def user_prompt_for(post):
    user_prompt = f"You are looking at a Reddit discussion titled:\n\n{post.title()}\n\n"
    user_prompt += "Below are the responses from various users. Analyze them according to the system prompt provided.\n"
    user_prompt += "Make sure your response is structured in Markdown with headers, lists, and tables as instructed.\n\n"
    user_prompt += post.comments(4000)
    return user_prompt


#### Generating Responses

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
def summarize(url):
    website = RedditPostScraper(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
# summarize("https://www.reddit.com/r/running/comments/1l77osa/pushing_through_a_run/")

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

#### Example Usage

In [ ]:
display_summary("https://www.reddit.com/r/running/comments/1l77osa/pushing_through_a_run/")

In [ ]:
display_summary("https://www.reddit.com/r/AskReddit/comments/1lam10k/how_do_you_feel_about_the_no_kings_protest/")

In [ ]:
display_summary("https://www.reddit.com/r/canada/comments/1laq8ok/donald_trump_is_a_convicted_felon_could_he_be/")